In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import timeit

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

# Load the training data set
data = pd.read_csv("../input/train.csv")
places = data['place_id']
features = data.drop(['place_id', 'row_id'], axis = 1)

#== DATA EXPLORATION
dta = data.copy()
dta['freq'] = dta.groupby('place_id')['x'].transform('count')

print(dta[:5])

In [ ]:
print( "row_id" )
%timeit dta[dta['row_id'] == 0]

In [ ]:

da = dta[['row_id', 'x', 'y']].copy()

# DATA VISUALIZATION
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from plotly import tools

plot3d = False
if plot3d:
    fig = plt.figure(figsize=(16,10))
else:
    halfSize = int(len(range(100, 140, 50)) / 2)
    f, axarr = plt.subplots(halfSize, halfSize) if (halfSize>1) else plt.subplots(1, 1)
    f.tight_layout()

import math
import itertools
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn import tree
from sklearn import preprocessing
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

iCount = 0
for i in range(100, 140, 50):
    print("i = ", i)
    
    # create grid
    start_index = 1
    a = 10
    N = i#math.sqrt(n)
    n = N*N#10000
    modVal = N - start_index + 1
    celldata = np.zeros((n,), dtype=[('index', 'i4'), ('row', 'i4'),('col', 'i4')])
    gridcell = pd.DataFrame(celldata)

    print( "N: ",N )

    cellValuesCombo = np.asarray(list(itertools.product(range(1, int(N)+1),range(1, int(N)+1))))
    gridcell['row'] = cellValuesCombo[:,0]
    gridcell['col'] = cellValuesCombo[:,1]
    gridcell['index'] = gridcell['row'] + (modVal*gridcell['col'])
    gridcell['index'] = gridcell['index'].astype(int)
    
    da['gridcell_x'] = (da['x']*N/a) + 1
    da['gridcell_x'] = da['gridcell_x'].astype(int)
    da['gridcell_y'] = (da['y']*N/a) + 1
    da['gridcell_y'] = da['gridcell_y'].astype(int)
    da['gridcell_index'] = da['gridcell_x'] + (modVal*da['gridcell_y'])
    da['gridcell_index'] = da['gridcell_index'].astype(int)
    da = da.drop(['gridcell_x', 'gridcell_y'], axis = 1)
    
    xy = da[da['gridcell_index'] == gridcell.iloc[0,0]]
    vw = dta[dta['row_id'].isin(xy['row_id'])].copy()
    vw['freq_grid'] = vw.groupby('place_id')['x'].transform('count')
    vw = vw[vw['freq_grid'] > 3].copy()
    
    print( "vw.shape: ",vw.shape )
    print( "vw unique places: ",len(vw['place_id'].unique()) )
    print()
    
    vw['hour'] = (vw['time'] / 60) % 24
    vw['hour'] = vw['hour'].astype(int)
    vw['weekday'] = (vw['time'] / (60*24)) % 7
    vw['weekday'] = vw['weekday'].astype(int)
    vw['month'] = (vw['time'] / (60*24*30)) % 12
    vw['month'] = vw['month'].astype(int)
    vw['year'] = vw['time'] / (60*24*365)
    vw['year'] = vw['year'].astype(int)
    vw['day'] = (vw['time'] / (60*24)) % 365
    vw['day'] = vw['day'].astype(int)
    
    """
    xy_sub = xy[['x', 'y', 'place_id', 'time', 'accuracy']]
    vw = xy[['x', 'y']]
    tu = xy[['x', 'y', 'time']]
    print( "xy: ",xy.shape )
    """


    # DATA VISUALIZATION
    #-- plotting x against y with colors for each place
    if plot3d:
        projCount = 221 + iCount
        ax = fig.add_subplot(projCount, projection='3d')
        ax.scatter(vw['x'], vw['y'], vw['hour'], c=vw['place_id'], linewidth=0.0)
    else:
        vw_sub = vw[['x', 'y']]

        #-- reduce dimensionality of features
        x_and_y = PCA(n_components=1).fit_transform(vw_sub)
        vw['x_and_y'] = x_and_y
        #tu_std = StandardScaler().fit_transform(tu)
        #x_and_y_and_time = PCA(n_components=2).fit_transform(tu_std)
    
        if (halfSize>0) :
            temp_x = (iCount % halfSize) #remainder
            temp_y = (iCount / halfSize) #quotient
            axarr[temp_x, temp_y].scatter(vw['x_and_y'], vw['time'], c=vw['place_id'], linewidth=0.0)#
        else:
            axarr.scatter(vw['x_and_y'], vw['hour'], c=vw['place_id'], linewidth=0.0)#
        
    iCount = iCount+1
    
    
    # ALGORITHM
    
    places = vw['place_id'].copy()
    #features = vw[['x', 'y', 'accuracy', 'hour', 'day']].copy()
    #features = vw[['x', 'y', 'hour']].copy()
    
    #"""
    features = vw[['x', 'y', 'hour']].copy()
    features['x_squared'] = features['x']**2
    features['y_squared'] = features['y']**2
    features['hour_squared'] = features['hour']**2
    #"""
    
    #"""
    #features['accuracy'] = vw['accuracy']
    #features['y_cubed'] = features['y']**3
    #features['hour_cubed'] = features['hour']**3
    #"""

    
    X = features
    y = places
    
    X_train, X_test, y_train, y_test = train_test_split(features, places, test_size=0.2, random_state=50)

    print( X_train[:5] )
    print()
    
    print( X_test[:5] )
    print()
    
    useLogReg = False
    useDTClf = False
    useRFClf = True
    if useLogReg:
        #"""
        logReg = LogisticRegression()
        logReg.fit(X_train, y_train)
        print ("\n\nlog reg score: %.3f", logReg.score(X_test, y_test))
        #"""
    
        #"""
        OVR = OneVsRestClassifier(LogisticRegression()).fit(X_train, y_train)
        print ("OVR accuracy score: %.3f", OVR.score(X_test, y_test))
        #"""
    
        #"""
        #X = features[:500]
        #y = places[:500]
        OVO = OneVsOneClassifier(LogisticRegression()).fit(X_train, y_train)
        print ("OVO accuracy score: %.3f", OVO.score(X_test, y_test))
        #"""
    elif useDTClf:
        #"""
        dtClf_model = tree.DecisionTreeClassifier()
        dtClf_model.fit(X_train, y_train)
        print ("\n\ndecision tree classifier score: %.3f", dtClf_model.score(X_test, y_test))
        #"""
    elif useRFClf:
        #"""
        rfClf_model = RandomForestClassifier(n_estimators=1000, # Number of trees
                                             max_features=2,    # Num features considered
                                             oob_score=True)    # Use OOB scoring*
        rfClf_model.fit(X_train, y_train)
        print ("\n\nrandom forest classifier score: %.3f", rfClf_model.score(X_test, y_test))
        print ("\n\nrandom forest classifier oob score: %.3f", rfClf_model.oob_score_)
        #"""

plt.show()

In [ ]:

print( dta[dta['row_id'] == 0])
